In [1]:
import cv2
from matplotlib import pyplot as plt
from natsort import natsorted
import numpy as np
import re
from glob import glob
from os.path import join, relpath
import os

def cut_start(dir_in):
    img1 = cv2.imread(dir_in)
    cropped1 = img1[1940:1945, 0:-1]
    cp2 = cv2.Canny(cropped1, 20, 20)
    indices = np.where(cp2 != [0])
    axis_x = []
    for coordinates in indices[1]:
        axis_x.append(coordinates)
        if len(axis_x) == 8:
            break
    array_i = np.array([axis_x[i] for i in range(1, len(axis_x) - 1) 
                        if min(abs(axis_x[i]-axis_x[i-1]), abs(axis_x[i+1]-axis_x[i])) >= 8])
    if len(array_i) > 0:
        sunset_t = int(array_i[0])
        return sunset_t
    else:
        return 20

def cut_show(dir_in):
    img = cv2.imread(dir_in)
    if cut_start(dir_in) <= img.shape[1] and cut_start(dir_in) >= 20:
        cutstart_left = cut_start(dir_in) - 20
        cutstart_right = cut_start(dir_in) + 20
    else:
        cutstart_left, cutstart_right = 0, 100
    cropped = img[1937:2266, cutstart_left:cutstart_right]
    #cv2.imshow('', cropped)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    return cv2.cvtColor(cropped, cv2.COLOR_RGB2GRAY)

def merge_file(pathoa, extension):
    files = [relpath(x,pathoa) for x in sorted(glob(join(pathoa,'*.{}'.format(extension))))]
    pngfiles = []
    i = 0
    while i < len(files):
        pngfiles.append(join(pathoa, files[i]))
        i += 1
    pngfiles = natsorted(pngfiles)
    return pngfiles

In [ ]:
#Single auto
#train set
date = '211112'
true_label = 'F'
location = ['A', 'D', 'P', 'N']

for i in range(len(location)):
    dir_in_a = f'raw_set/{date}{location[i]}_{true_label}.PNG'
    dir_out_a = f'pro_set/{date}{location[i]}_{true_label}.jpg'

    cropped_a = cut_show(dir_in_a)
    cv2.imwrite(dir_out_a, cropped_a)
    cropped_c1 = cv2.imread(dir_out_a)
    cv2.normalize(cropped_c1, cropped_c1, 0, 255, cv2.NORM_MINMAX)
    cv2.imwrite(dir_out_a, cropped_c1)

In [2]:
#test
date = '211117'
true_label = 'F'
location = ['A', 'D', 'P', 'N']

for i in range(len(location)):
    
    dir_in_a = f'raw_set/{date}{location[i]}_{true_label}.PNG' #Win
    dir_out_a = f'test_set/{date}{location[i]}_{true_label}.jpg' #Win
    
    cropped_a = cut_show(dir_in_a)
    cv2.imwrite(dir_out_a, cropped_a)
    cropped_c1 = cv2.imread(dir_out_a)
    cv2.normalize(cropped_c1, cropped_c1, 0, 255, cv2.NORM_MINMAX)
    cv2.imwrite(dir_out_a, cropped_c1)

In [ ]:
pattern = '[/\\\.]'
dir_out_list = []
pngfiles = merge_file('raw_set', 'PNG')
for i in range(len(pngfiles)):
    dir_in = pngfiles[i]
    img_na = re.split(pattern, dir_in)[-2]
    dir_out = 'pro_set/' + img_na + '.jpg'
    cropped = cut_show(dir_in)
    cv2.normalize(cropped, cropped, 0, 255, cv2.NORM_MINMAX)
    cv2.imwrite(dir_out, cropped)

In [ ]:
#Manually Manual
dir_in_m = 'raw_set/211030A_T.PNG'
dir_out_m = 'pro_set/211030A_T.jpg'
def cut_show_manual(dir_in_m, cutstart_left, cutstart_above = 1937):
    img = cv2.imread(dir_in_m)
    cutstart_below = cutstart_above + 329
    cutstart_right = cutstart_left + 40
    cropped = img[cutstart_above:cutstart_below, cutstart_left:cutstart_right]
    cv2.imshow('', cropped)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return cv2.cvtColor(cropped, cv2.COLOR_RGB2GRAY)

#1592
cropped_m = cut_show_manual(dir_in_m, cutstart_left = 899, cutstart_above = 1937)
cv2.normalize(cropped_m, cropped_m, 0, 255, cv2.NORM_MINMAX)
cv2.imwrite(dir_out_m, cropped_m)

In [ ]:
sunset_t = int(np.array([axis_x[i] for i in range(1, len(axis_x) - 1) 
                         if min(abs(axis_x[i]-axis_x[i-1]), abs(axis_x[i+1]-axis_x[i])) >= 10])[0])

845